In [1]:
import sys

sys.path.append('../..')

In [2]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1/360.
mpor = 15*step

#Discount

In [3]:
from finance.discountfactor import ConstantRateDiscountFactor 

maturity = 5.
r = 0.02

discount = ConstantRateDiscountFactor(r)

# Exponential distrib

In [4]:
from scipy.stats import expon
 
lbda = 2.0 / maturity
exp_distrib = expon(loc=0, scale=1.0/lbda)

# Underlying

In [5]:
import numpy as np
from scipy.stats import norm
from maths.montecarlo.process.brownianmotion import BrownianMotion

time_grid = BrownianMotion.generate_time_grid(0, maturity, step)

x_0 = [100]
mu_s = [6]
sigma_s = [15]

b = BrownianMotion(x_0, mu_s, sigma_s, time_grid)

for i, (x0, m_, s_) in enumerate(zip(x_0, mu_s, sigma_s)):
    print "P(S_T^%d < 0) = %s"%(i, norm.cdf(-(x0 + m_ * maturity)/(s_*np.sqrt(maturity))))

P(S_T^0 < 0) = 5.31263106722e-05


# Derivative

In [6]:
from finance.products.european.assetswap import (
    SwapContract,
)

delta = 0.5

swap_dates = SwapContract.generate_payment_dates(0, maturity, delta)
swap = SwapContract(b, discount, swap_dates)

price_0 = swap.price(0.)

print swap
print "\nPrice swap at t=0 = ",price_0

Swap contract of maturity T = 5 years, over S^0 with strike K = 104.560, paying at {0.00, 0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00}

Price swap at t=0 =  -5.68434188608e-14


# Exposure

In [7]:
from risk.exposure import EuropeanQuantileBrownianExposure

index = swap.underlying_index
drift = mu_s[index]
vol = sigma_s[index]
exposure = EuropeanQuantileBrownianExposure(swap, discount, drift, vol)

# Load intensities of obligors

In [8]:
import dill

raw_intensities = None
with open('../intensities/intensities.pkl', 'rb') as f:
    raw_intensities = dill.load(f)
    
raw_intensities.keys()

['groups', 'idio']

In [9]:
raw_idios = np.array(raw_intensities['idio'])
raw_groups = np.array(raw_intensities['groups'])

In [10]:
subsets = []
hazard_rates = []
pillars = []

for d_idio in raw_idios:
    subsets.append(d_idio['id'])
    hazard_rates.append(d_idio['hazard_rates'])
    pillars.append(d_idio['pillars'])
    
for d_groups in raw_groups:    
    subsets.append(d_groups['ids'])
    hazard_rates.append(d_groups['hazard_rates'])
    pillars.append(d_groups['pillars'])

# IDs parameters

In [11]:
bank_index = 0
counterparty_index = 1

# Portfolio construction

In [12]:
p_fixed = 100.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

In [13]:
from finance.portfolio import EquilibratedPortfolio

pos = EquilibratedPortfolio.generate_1_vs_all_positions(0, True, raw_idios.size)
positions = np.array(pos*notional).reshape(pos.size, 1)

port = EquilibratedPortfolio(positions, [swap], [exposure])

#Copula

In [14]:
from maths.copula.marshallolkin import StepWiseIntensitiesMarshallOlkinCopula

copula = StepWiseIntensitiesMarshallOlkinCopula(subsets, hazard_rates, pillars)